# Extract 8 regions

We're going to extract the 8 regions of interest (ROIs) from the _cis-eQTL_ data.


## Load packages

Here we load the necessary packages.


In [1]:
# Function to check for installation of required packages
def check_install_package(package_name):
    try:
        importlib.import_module(package_name)
    except ImportError:
        print(f'{package_name} is not installed. Install and try again...')
        # subprocess.check_call(['pip', 'install', package_name])

import os
import glob
import importlib
import subprocess
import sys

# get date and time
from datetime import datetime

# Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool
# check_install_package('pandas')
import pandas as pd

# pyarrow is supperior to loading parquet files
# check_install_package('pyarrow')
import pyarrow as pa
import pyarrow.parquet as pq

# polars is a fast dataframe library
# check_install_package('polars')
import polars as pl

# for statistical analysis
# check_install_package('scipy')
from scipy import stats
import numpy as np



We also want some variables set.


In [2]:
# Create directories for the GWAS data and the reference data
from subprocess import check_output

# set some general defaults
PGC_HITS = "PGC"

POPULATION = "EUR"

# general plotting directory
PLOTS_loc = "PLOTS"

# location to put molQTL results
molQTL_loc = "molQTL_results"

# Check if the directory exists
if not os.path.exists(molQTL_loc):
    # If it doesn't exist, create it
    os.makedirs(molQTL_loc)

# Check if the directory exists
if not os.path.exists(PLOTS_loc):
    # If it doesn't exist, create it
    os.makedirs(PLOTS_loc)

# # regional association plots directory
# REG_PLOTS_loc = PLOTS_loc + "/Regional_Association_Plots"

# # Check if the directory exists
# if not os.path.exists(REG_PLOTS_loc):
#     # If it doesn't exist, create it
#     os.makedirs(REG_PLOTS_loc)

# Reference data directory
REF_loc = "/Users/slaan3/PLINK/references"
print("Checking contents of the reference directory:")
print(check_output(["ls", os.path.join(REF_loc)]).decode("utf8"))

# GWAS data directory
GD_loc = "/Users/slaan3/Library/CloudStorage/GoogleDrive-s.w.vanderlaan@gmail.com/My Drive/Genomics/#Projects/TO_AITION/MR CVD MDD/GWAS"
print("Checking contents of the Google Drive directory:")
print(check_output(["ls", os.path.join(GD_loc)]).decode("utf8"))

# molQTL data directory
NOM_CIS_EQTL_loc = "/Users/slaan3/git/CirculatoryHealth/molqtl/results/version1_aernas1_firstrun/nom_cis_eqtl"
print("Checking contents of the nominal cis-eQTL data directory:")
print(check_output(["ls", os.path.join(NOM_CIS_EQTL_loc)]).decode("utf8"))

PERM_CIS_MQTL_loc = "/Users/slaan3/git/CirculatoryHealth/molqtl/results/perm_cis_mqtl"
print("Checking contents of the permuted cis-mQTL data directory:")
print(check_output(["ls", os.path.join(PERM_CIS_MQTL_loc)]).decode("utf8"))

PERM_TRANS_EQTL_loc = "/Users/slaan3/git/CirculatoryHealth/molqtl/results/version1_aernas1_firstrun/perm_trans_eqtl"
print("Checking contents of the permuted trans-eQTL data directory:")
print(check_output(["ls", os.path.join(PERM_TRANS_EQTL_loc)]).decode("utf8"))

PERM_TRANS_MQTL_loc = (
    "/Users/slaan3/git/CirculatoryHealth/molqtl/results/perm_trans_mqtl"
)
print("Checking contents of the permuted trans-eQTL data directory:")
print(check_output(["ls", os.path.join(PERM_TRANS_MQTL_loc)]).decode("utf8"))

Checking contents of the reference directory:
1000G
GoNL
HRC_r1_1_2016
HRCr11_1000Gp3v5
dbSNP
fasta
refgenie_genomes
tcga

Checking contents of the Google Drive directory:
CAC
CIMT
GIGASTROKE
MILLIONHEARTS
PGC

Checking contents of the nominal cis-eQTL data directory:
README.md
tensorqtl_cis_nominal_chr01.cis_qtl_pairs.chr01.parquet
tensorqtl_cis_nominal_chr02.cis_qtl_pairs.chr02.parquet
tensorqtl_cis_nominal_chr03.cis_qtl_pairs.chr03.parquet
tensorqtl_cis_nominal_chr04.cis_qtl_pairs.chr04.parquet
tensorqtl_cis_nominal_chr05.cis_qtl_pairs.chr05.parquet
tensorqtl_cis_nominal_chr06.cis_qtl_pairs.chr06.parquet
tensorqtl_cis_nominal_chr07.cis_qtl_pairs.chr07.parquet
tensorqtl_cis_nominal_chr08.cis_qtl_pairs.chr08.parquet
tensorqtl_cis_nominal_chr09.cis_qtl_pairs.chr09.parquet
tensorqtl_cis_nominal_chr10.cis_qtl_pairs.chr10.parquet
tensorqtl_cis_nominal_chr11.cis_qtl_pairs.chr11.parquet
tensorqtl_cis_nominal_chr12.cis_qtl_pairs.chr12.parquet
tensorqtl_cis_nominal_chr13.cis_qtl_pairs.chr13.p

## Load data

We are going to need the list of ROIs and the annotated cis-eQTL data.


### Regions-of-interest

Let's load the 8 regions we are interested in.


In [3]:
# polars.read_excel(
# source: str | BytesIO | Path | BinaryIO | bytes,
# *,
# sheet_id: None = None,
# sheet_name: str,
# engine: Literal['xlsx2csv', 'openpyxl', 'pyxlsb'] | None = None,
# xlsx2csv_options: dict[str, Any] | None = None,
# read_csv_options: dict[str, Any] | None = None,
# schema_overrides: SchemaDict | None = None,
# raise_if_empty: bool = True,
# )

target_regions = pl.read_csv(
    source=os.path.join("targets/regions.csv")
)


In [4]:
target_regions

leadSNP,chromosome,lower_position,upper_position
str,i64,i64,i64
"""7:12263538""",7,11263538,13263538
"""19:4044579""",19,3044579,5044579
"""15:38843887""",15,37843887,39843887
"""7:114059156""",7,113059156,115059156
"""7:2760750""",7,1760750,3760750
"""7:1937261""",7,937261,2937261
"""2:165045101""",2,164045101,166045101
"""2:164915279""",2,163915279,165915279


### _cis_-eQTL data

We previously parsed the _cis_-eQTL data into a `.parquet` file which includes annotated results. For details check `https://github.com/CirculatoryHealth/molqtl` GitHub repository.

Here we'll simply load the annotated data.


In [15]:
# read in data
# https://stackoverflow.com/questions/33813815/how-to-read-a-parquet-file-into-pandas-dataframe

import polars as pl

# annotated cis-eQTLs
sumstats_nom_cis_eqtl = pl.read_parquet(
    source=os.path.join(
        NOM_CIS_EQTL_loc, "tensorqtl_nominal_cis_qtl_pairs.annot.parquet"
    )
)


In [16]:
sumstats_nom_cis_eqtl

EnsemblID,VariantID,tss_distance,CAF_eQTL,ma_samples,ma_count,pval_nominal,Beta,SE,chromosome,position,OtherAlleleA,CodedAlleleB,CAF,AltID,Source,AverageMaximumPosteriorCall,Info,AA_N,AB_N,BB_N,TotalN,MAF,MissingDataProportion,HWE_P
str,str,i32,f32,i32,i32,f64,f32,f32,i64,i64,str,str,f64,str,str,f64,f64,f64,f64,f64,i64,f64,f64,f64
"""ENSG00000187634""","""1:693731""",-240519,0.134185,153,168,0.881492,0.005953,0.039917,1,693731,"""A""","""G""",0.137241,"""rs12238997""","""HRCr11""",0.902046,0.624501,1637.01,451.103,35.569,2124,0.122957,0.000075,0.544841
"""ENSG00000187634""","""1:714596""",-219654,0.038339,48,48,0.834746,-0.015369,0.073635,1,714596,"""T""","""C""",0.0327213,"""rs149887893""","""HRCr11""",0.968278,0.577564,1987.73,135.495,0.677,2124,0.0322165,0.000024,0.272504
"""ENSG00000187634""","""1:715367""",-218883,0.038339,48,48,0.834746,-0.015369,0.073635,1,715367,"""A""","""G""",0.0324859,"""rs12184277""","""HRCr11""",0.975207,0.67196,1976.85,146.512,0.556,2124,0.0347528,0.000019,0.176551
"""ENSG00000187634""","""1:717485""",-216765,0.038339,48,48,0.834746,-0.015369,0.073635,1,717485,"""C""","""A""",0.0324859,"""rs12184279""","""HRCr11""",0.975183,0.670147,1978.35,145.041,0.517,2124,0.0343883,0.000022,0.174817
"""ENSG00000187634""","""1:720381""",-213869,0.038339,48,48,0.834746,-0.015369,0.073635,1,720381,"""G""","""T""",0.0327213,"""rs116801199""","""HRCr11""",0.974476,0.667536,1973.72,149.503,0.691,2124,0.0355205,0.00002,0.110452
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ENSG00000079974""","""22:51220249""",444664,0.035942,43,45,0.863512,-0.004492,0.026119,22,51220249,"""A""","""T""",0.0442561,"""rs9616977""","""1000Gp3v5""",0.913151,0.403798,1832.6,278.562,12.692,2124,0.0715553,0.000034,0.622809
"""ENSG00000079974""","""22:51221190""",445605,0.035144,42,44,0.671267,0.011112,0.026169,22,51221190,"""G""","""A""",0.0400188,"""rs369304721""","""HRCr11""",0.933398,0.443438,1911.83,204.73,7.373,2124,0.0516673,0.000015,0.500544
"""ENSG00000079974""","""22:51221731""",446146,0.035942,43,45,0.863512,-0.004492,0.026119,22,51221731,"""T""","""C""",0.0442561,"""rs115055839""","""HRCr11""",0.911894,0.405944,1826.36,284.126,13.448,2124,0.0732185,0.000016,0.523538


### _cis_-mQTL data

We previously parsed the _cis_-mQTL data into a `.parquet` file which includes annotated results. For details check `https://github.com/CirculatoryHealth/molqtl` GitHub repository.

Here we'll simply load the annotated data.


In [52]:
# read in data
# https://stackoverflow.com/questions/33813815/how-to-read-a-parquet-file-into-pandas-dataframe

import polars as pl

# Define the schema for your columns, e.g., `num_var` as an integer (or other types if necessary)
schema_overrides = {
    "num_var": pl.Int64  # or pl.Float64 if you expect floats, or pl.Utf8 if it's text data
}
# annotated cis-mQTLs
sumstats_nom_cis_mqtl = pl.read_csv(
    source=os.path.join(PERM_CIS_MQTL_loc, "tensormqtl.perm_cis_mqtl.txt"),
    separator='\t',
    schema_overrides=schema_overrides,
    infer_schema_length=10000,  # Increase if needed
    ignore_errors=True  # Skips rows with parsing errors
)


In [53]:
sumstats_nom_cis_mqtl

phenotype_id,num_var,beta_shape1,beta_shape2,true_df,pval_true_df,variant_id,tss_distance,ma_samples,ma_count,af,pval_nominal,slope,slope_se,pval_perm,pval_beta,qval,pval_nominal_threshold
str,i64,f64,f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64
"""cg21870274""",309,1.01037,15.0882,351.533,0.0495503,"""1:752307""",682715,15,15,0.0170068,0.036876,-0.406806,0.194254,0.531247,0.530703,0.468602,0.001232
"""cg08258224""",731,1.03658,40.9219,342.718,0.101636,"""1:1636400""",836316,37,38,0.0430839,0.0780652,-0.160865,0.0910595,0.984102,0.986523,0.605861,0.000509
"""cg18147296""",750,1.02594,43.8044,350.307,0.000046,"""1:771410""",-41130,130,141,0.840136,0.000014,-0.195278,0.0444456,0.0017,0.001693,0.006203,0.000454
"""cg13938959""",787,1.00503,41.991,346.635,0.0160055,"""1:800193""",-33991,7,7,0.007937,0.009935,0.568674,0.219518,0.49525,0.48971,0.45302,0.000433
"""cg12445832""",787,1.01013,42.2714,344.888,0.003059,"""1:1706886""",872590,44,46,0.0521542,0.001483,-0.25942,0.08106,0.118388,0.118428,0.211842,0.000439
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""cg19004771""",2979,1.04971,202.825,323.792,0.000565,"""22:51181759""",8693,100,108,0.877551,0.000134,-0.238654,0.0618922,0.0933907,0.0950843,0.152963,0.000135
"""cg20569369""",2979,1.04484,216.682,329.092,0.00114,"""22:50519640""",-654007,25,25,0.0283447,0.000352,-0.456097,0.126513,0.20348,0.201155,0.249547,0.000124
"""cg26034629""",2975,1.03817,214.838,329.486,0.003815,"""22:50913182""",-262183,75,79,0.0895692,0.001493,0.316939,0.0990935,0.549345,0.542439,0.423997,0.000122


## Extract _cis_-eQTL data

Now we're ready to extract the data for each region in one table.


In [20]:
import polars as pl

# Initialize an empty DataFrame with the same schema as sumstats_nom_cis_eqtl
filtered_results = pl.DataFrame(schema=sumstats_nom_cis_eqtl.schema)

# Loop through each row of df2 and filter df1 based on chromosome and position range
for row in target_regions.iter_rows(named=True):
    chrom = row['chromosome']
    lower_pos = row['lower_position']
    upper_pos = row['upper_position']
    
    # Filter df1 based on the chromosome and position range using polars
    filtered = sumstats_nom_cis_eqtl.filter(
        (sumstats_nom_cis_eqtl['chromosome'] == chrom) & 
        (sumstats_nom_cis_eqtl['position'] >= lower_pos) & 
        (sumstats_nom_cis_eqtl['position'] <= upper_pos)
    )
    
    # Append the filtered rows to the result DataFrame
    filtered_results = filtered_results.vstack(filtered)

# Show the final filtered DataFrame
print(filtered_results)


shape: (621_506, 25)
┌────────────┬────────────┬────────────┬──────────┬───┬────────┬───────────┬────────────┬──────────┐
│ EnsemblID  ┆ VariantID  ┆ tss_distan ┆ CAF_eQTL ┆ … ┆ TotalN ┆ MAF       ┆ MissingDat ┆ HWE_P    │
│ ---        ┆ ---        ┆ ce         ┆ ---      ┆   ┆ ---    ┆ ---       ┆ aProportio ┆ ---      │
│ str        ┆ str        ┆ ---        ┆ f32      ┆   ┆ i64    ┆ f64       ┆ n          ┆ f64      │
│            ┆            ┆ i32        ┆          ┆   ┆        ┆           ┆ ---        ┆          │
│            ┆            ┆            ┆          ┆   ┆        ┆           ┆ f64        ┆          │
╞════════════╪════════════╪════════════╪══════════╪═══╪════════╪═══════════╪════════════╪══════════╡
│ ENSG000001 ┆ 7:11263538 ┆ 327489     ┆ 0.071086 ┆ … ┆ 2124   ┆ 0.0684027 ┆ 0.000001   ┆ 0.610073 │
│ 89043      ┆            ┆            ┆          ┆   ┆        ┆           ┆            ┆          │
│ ENSG000001 ┆ 7:11264536 ┆ 328487     ┆ 0.047923 ┆ … ┆ 2124   ┆ 0.053

In [21]:
# Count the number of rows in the filtered_results DataFrame
num_rows = filtered_results.shape[0]

# Print the number of rows
print(f"Number of rows in filtered_results: {num_rows}")


Number of rows in filtered_results: 621506


In [22]:
del sumstats_nom_cis_eqtl

### Annotation

Let's also annotate this table with more information.


In [23]:
import mygene
import pandas as pd
import polars as pl

# Initialize mygene client
mg = mygene.MyGeneInfo()

# Convert the polars DataFrame (filtered_results) to a pandas DataFrame
filtered_results_pd = filtered_results.to_pandas()

# Get the unique Ensembl IDs from the polars DataFrame (filtered_results)
ensembl_ids = filtered_results['EnsemblID'].unique().to_list()

# Query mygene to get additional gene information (e.g., symbol, name, etc.)
gene_info = mg.querymany(ensembl_ids, 
                         scopes='ensembl.gene', 
                         fields='symbol,name,alias,type_of_gene,map_location,genomic_pos,entrezgene,HGNC,summary', 
                         species='human')

# Convert the gene info to a pandas DataFrame
gene_info_df = pd.DataFrame(gene_info)

# Select relevant columns
gene_info_df = gene_info_df[['query', 'symbol', 'name', 'alias', 'type_of_gene', 'map_location', 'genomic_pos', 'entrezgene', 'HGNC', 'summary']]

# Convert the pandas DataFrame back to polars DataFrame
gene_info_df

,query,symbol,name,alias,type_of_gene,map_location,genomic_pos,entrezgene,HGNC,summary
0,ENSG00000105248,YJU2,YJU2 splicing factor homolog,CCDC94,protein-coding,19p13.3,"{'chr': '19', 'end': 4269088, 'ensemblgene': '...",55702,25518,Predicted to enable metal ion binding activity...
1,ENSG00000188549,CCDC9B,coiled-coil domain containing 9B,C15orf52,protein-coding,15q15.1,"{'chr': '15', 'end': 40340939, 'ensemblgene': ...",388115,33488,Enables RNA binding activity. [provided by All...
2,ENSG00000157778,PSMG3,proteasome assembly chaperone 3,"[C7orf48, PAC3, Pba3]",protein-coding,7p22.3,"{'chr': '7', 'end': 1571005, 'ensemblgene': 'E...",84262,22420,Enables molecular adaptor activity. Involved i...
3,ENSG00000146530,VWDE,von Willebrand factor D and EGF domains,NaN,protein-coding,7p21.3,"{'chr': '7', 'end': 12403941, 'ensemblgene': '...",221806,21897,Predicted to enable signaling receptor binding...
4,ENSG00000106268,NUDT1,nudix hydrolase 1,MTH1,protein-coding,7p22.3,"{'chr': '7', 'end': 2251146, 'ensemblgene': 'E...",4521,8048,Misincorporation of oxidized nucleoside tripho...
...,...,...,...,...,...,...,...,...,...,...
171,ENSG00000140320,BAHD1,bromo adjacent homology domain containing 1,NaN,protein-coding,15q15.1,"{'chr': '15', 'end': 40468236, 'ensemblgene': ...",22893,29153,Enables chromatin binding activity. Involved i...
172,ENSG00000172000,ZNF556,zinc finger protein 556,NaN,protein-coding,19p13.3,"{'chr': '19', 'end': 2883445, 'ensemblgene': '...",80032,25669,Predicted to enable DNA-binding transcription ...
173,ENSG00000073067,CYP2W1,cytochrome P450 family 2 subfamily W member 1,NaN,protein-coding,7p22.3,"{'chr': '7', 'end': 989640, 'ensemblgene': 'EN...",54905,20243,This gene encodes a member of the cytochrome P...
174,ENSG00000005108,THSD7A,thrombospondin type 1 domain containing 7A,NaN,protein-coding,7p21.3,"{'chr': '7', 'end': 11832198, 'ensemblgene': '...",221981,22207,The protein encoded by this gene is found almo...


In [24]:
# Merge the original pandas DataFrame (filtered_results_pd) with the gene information DataFrame on 'EnsemblID'
filtered_results_annotated = filtered_results_pd.merge(gene_info_df, left_on='EnsemblID', right_on='query', how='left')

# Drop the 'query' column (since it's the same as 'EnsemblID')
filtered_results_annotated.drop(columns=['query'], inplace=True)

# Display the annotated DataFrame
print(filtered_results_annotated)

              EnsemblID    VariantID  tss_distance  CAF_eQTL  ma_samples  \
0       ENSG00000189043   7:11263538        327489  0.071086          84   
1       ENSG00000189043   7:11264536        328487  0.047923          56   
2       ENSG00000189043   7:11264670        328621  0.075879          88   
3       ENSG00000189043   7:11264692        328643  0.521565         455   
4       ENSG00000189043   7:11265421        329372  0.598243         404   
...                 ...          ...           ...       ...         ...   
621501  ENSG00000136546  2:165913564       -593964  0.646965         362   
621502  ENSG00000136546  2:165914241       -593287  0.162939         186   
621503  ENSG00000136546  2:165914353       -593175  0.227636         251   
621504  ENSG00000136546  2:165914416       -593112  0.234026         259   
621505  ENSG00000136546  2:165914802       -592726  0.137380         155   

        ma_count  pval_nominal      Beta        SE  chromosome  ...     HWE_P  \
0     

In [26]:
# Count the number of rows in filtered_results_annotated_pd
num_rows = filtered_results_annotated.shape[0]

# Print the number of rows
print(f"Number of rows in filtered_results_annotated_pd: {num_rows}")


Number of rows in filtered_results_annotated_pd: 621506


In [27]:
# Display the headers of the DataFrame
print(filtered_results_annotated.columns)


Index(['EnsemblID', 'VariantID', 'tss_distance', 'CAF_eQTL', 'ma_samples',
       'ma_count', 'pval_nominal', 'Beta', 'SE', 'chromosome', 'position',
       'OtherAlleleA', 'CodedAlleleB', 'CAF', 'AltID', 'Source',
       'AverageMaximumPosteriorCall', 'Info', 'AA_N', 'AB_N', 'BB_N', 'TotalN',
       'MAF', 'MissingDataProportion', 'HWE_P', 'symbol', 'name', 'alias',
       'type_of_gene', 'map_location', 'genomic_pos', 'entrezgene', 'HGNC',
       'summary'],
      dtype='object')


### Saving

Let's save the annotated table.


In [28]:
# Specify the order of columns if needed
columns_order = ['EnsemblID', 'symbol', 
'VariantID', 'tss_distance', 'CAF_eQTL', 'ma_samples',
'ma_count', 'pval_nominal', 'Beta', 'SE', 'chromosome', 'position',
'OtherAlleleA', 'CodedAlleleB', 'CAF', 'AltID', 'Source',
'AverageMaximumPosteriorCall', 'Info', 'AA_N', 'AB_N', 'BB_N', 'TotalN',
'MAF', 'MissingDataProportion', 'HWE_P', 
'name', 'alias', 'type_of_gene', 'map_location', 'genomic_pos', 'entrezgene', 'HGNC', 'summary']
# Ensure these columns exist in your DataFrame

# Save the DataFrame with specified columns
filtered_results_annotated.to_csv(
    os.path.join(
        molQTL_loc, 'target_regions.nom_cis_eqtl_annotated.txt.gz'),
    sep='\t',
    index=False,
    compression='gzip',
    columns=columns_order
)


In [ ]:
del filtered_results # remove the filtered_results DataFrame from memory
del filtered_results_annotated # remove the filtered_results_annotated DataFrame from memory
del filtered # remove the filtered DataFrame from memory
del filtered_results_pd # remove the filtered_results_pd DataFrame from memory

## Extract _cis_-mQTL data

Now we're ready to extract the data for each region in one table.


In [54]:
# Split variant_id into chromosome and position directly
sumstats_nom_cis_mqtl = sumstats_nom_cis_mqtl.with_columns([
    pl.col("variant_id"),  # Retain the original column
    pl.col("variant_id").str.extract(r"(\d+):", 1).alias("chromosome"),
    pl.col("variant_id").str.extract(r":(\d+)", 1).cast(pl.Int64).alias("position")
])

# Show the resulting DataFrame
print(sumstats_nom_cis_mqtl)

shape: (268_375, 20)
┌────────────┬─────────┬────────────┬────────────┬───┬──────────┬───────────┬───────────┬──────────┐
│ phenotype_ ┆ num_var ┆ beta_shape ┆ beta_shape ┆ … ┆ qval     ┆ pval_nomi ┆ chromosom ┆ position │
│ id         ┆ ---     ┆ 1          ┆ 2          ┆   ┆ ---      ┆ nal_thres ┆ e         ┆ ---      │
│ ---        ┆ i64     ┆ ---        ┆ ---        ┆   ┆ f64      ┆ hold      ┆ ---       ┆ i64      │
│ str        ┆         ┆ f64        ┆ f64        ┆   ┆          ┆ ---       ┆ str       ┆          │
│            ┆         ┆            ┆            ┆   ┆          ┆ f64       ┆           ┆          │
╞════════════╪═════════╪════════════╪════════════╪═══╪══════════╪═══════════╪═══════════╪══════════╡
│ cg21870274 ┆ 309     ┆ 1.01037    ┆ 15.0882    ┆ … ┆ 0.468602 ┆ 0.001232  ┆ 1         ┆ 752307   │
│ cg08258224 ┆ 731     ┆ 1.03658    ┆ 40.9219    ┆ … ┆ 0.605861 ┆ 0.000509  ┆ 1         ┆ 1636400  │
│ cg18147296 ┆ 750     ┆ 1.02594    ┆ 43.8044    ┆ … ┆ 0.006203 ┆ 0.00

In [55]:
import polars as pl

# Initialize an empty DataFrame with the same schema as sumstats_nom_cis_mqtl
filtered_results = pl.DataFrame(schema=sumstats_nom_cis_mqtl.schema)

# Loop through each row of df2 and filter df1 based on chromosome and position range
for row in target_regions.iter_rows(named=True):
    chrom = row['chromosome']
    lower_pos = row['lower_position']
    upper_pos = row['upper_position']
    
    # Filter df1 based on the chromosome and position range using polars
    filtered = sumstats_nom_cis_mqtl.filter(
        (sumstats_nom_cis_mqtl['chromosome'] == chrom) & 
        (sumstats_nom_cis_mqtl['position'] >= lower_pos) & 
        (sumstats_nom_cis_mqtl['position'] <= upper_pos)
    )
    
    # Append the filtered rows to the result DataFrame
    filtered_results = filtered_results.vstack(filtered)

# Show the final filtered DataFrame
print(filtered_results)


shape: (3_558, 20)
┌────────────┬─────────┬────────────┬───────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ phenotype_ ┆ num_var ┆ beta_shape ┆ beta_shap ┆ … ┆ qval     ┆ pval_nomi ┆ chromosom ┆ position  │
│ id         ┆ ---     ┆ 1          ┆ e2        ┆   ┆ ---      ┆ nal_thres ┆ e         ┆ ---       │
│ ---        ┆ i64     ┆ ---        ┆ ---       ┆   ┆ f64      ┆ hold      ┆ ---       ┆ i64       │
│ str        ┆         ┆ f64        ┆ f64       ┆   ┆          ┆ ---       ┆ str       ┆           │
│            ┆         ┆            ┆           ┆   ┆          ┆ f64       ┆           ┆           │
╞════════════╪═════════╪════════════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ cg10940374 ┆ 8142    ┆ 1.05747    ┆ 430.204   ┆ … ┆ 0.108641 ┆ 0.000057  ┆ 7         ┆ 11621877  │
│ cg23711939 ┆ 8140    ┆ 1.05286    ┆ 415.566   ┆ … ┆ 0.234493 ┆ 0.000058  ┆ 7         ┆ 11902812  │
│ cg23083824 ┆ 8086    ┆ 1.05272    ┆ 419.446   ┆ … ┆ 0.353023 ┆ 0.00005

In [56]:
# Count the number of rows in the filtered_results DataFrame
num_rows = filtered_results.shape[0]

# Print the number of rows
print(f"Number of rows in filtered_results: {num_rows}")


Number of rows in filtered_results: 3558


In [57]:
del sumstats_nom_cis_mqtl

### Annotation

Let's also annotate this table with more information.


In [58]:
import pandas as pd
from rpy2.robjects import r, pandas2ri
import rpy2.robjects.packages as rpackages

# Enable the automatic conversion between R and pandas DataFrames
pandas2ri.activate()

# Load the Illumina annotation package in R
illumina_pkg = rpackages.importr('IlluminaHumanMethylation450kanno.ilmn12.hg19')

# Load and convert the annotation data to an R data frame
r('library(IlluminaHumanMethylation450kanno.ilmn12.hg19)')
r('annotation_df <- as.data.frame(getAnnotation(IlluminaHumanMethylation450kanno.ilmn12.hg19))')

# Fetch the data frame into Python
annotation_df = pandas2ri.rpy2py(r['annotation_df'])

# Assuming 'phenotype_id' in filtered_results corresponds to CpG IDs in annotation_df
filtered_results_pd = filtered_results.to_pandas()

# Merge filtered_results with the annotation data
filtered_results_annotated = filtered_results_pd.merge(annotation_df, left_on='phenotype_id', right_index=True, how='left')

# Display the merged data
print(filtered_results_annotated)

     phenotype_id  num_var  beta_shape1  beta_shape2  true_df  pval_true_df  \
0      cg10940374     8142      1.05747      430.204  329.506      0.000144   
1      cg23711939     8140      1.05286      415.566  327.662      0.000447   
2      cg23083824     8086      1.05272      419.446  329.860      0.000984   
3      cg07287563     8085      1.06727      369.628  320.193      0.009988   
4      cg22477374     8085      1.03771      391.770  325.199      0.000866   
...           ...      ...          ...          ...      ...           ...   
3553   cg19049344     4736      1.06155      257.846  330.619      0.002345   
3554   cg00508771     4751      1.05287      242.645  327.344      0.000858   
3555   cg07322293     5583      1.04651      261.410  328.283      0.002620   
3556   cg11219022     5459      1.03068      268.048  332.278      0.001043   
3557   cg13268671     5458      1.04332      254.938  328.934      0.013409   

       variant_id  tss_distance  ma_samples  ma_cou

In [59]:
# Count the number of rows in filtered_results_annotated_pd
num_rows = filtered_results_annotated.shape[0]

# Print the number of rows
print(f"Number of rows in filtered_results_annotated_pd: {num_rows}")


Number of rows in filtered_results_annotated_pd: 3558


In [60]:
# Display the headers of the DataFrame
print(filtered_results_annotated.columns)


Index(['phenotype_id', 'num_var', 'beta_shape1', 'beta_shape2', 'true_df',
       'pval_true_df', 'variant_id', 'tss_distance', 'ma_samples', 'ma_count',
       'af', 'pval_nominal', 'slope', 'slope_se', 'pval_perm', 'pval_beta',
       'qval', 'pval_nominal_threshold', 'chromosome', 'position', 'chr',
       'pos', 'strand', 'Name', 'AddressA', 'AddressB', 'ProbeSeqA',
       'ProbeSeqB', 'Type', 'NextBase', 'Color', 'Probe_rs', 'Probe_maf',
       'CpG_rs', 'CpG_maf', 'SBE_rs', 'SBE_maf', 'Islands_Name',
       'Relation_to_Island', 'Forward_Sequence', 'SourceSeq', 'Random_Loci',
       'Methyl27_Loci', 'UCSC_RefGene_Name', 'UCSC_RefGene_Accession',
       'UCSC_RefGene_Group', 'Phantom', 'DMR', 'Enhancer', 'HMM_Island',
       'Regulatory_Feature_Name', 'Regulatory_Feature_Group', 'DHS'],
      dtype='object')


### Saving

Let's save the annotated table.


In [61]:
# Specify the order of columns if needed
columns_order = ['phenotype_id', 'num_var', 'beta_shape1', 'beta_shape2', 'true_df',
       'pval_true_df', 'variant_id', 'chromosome', 'position', 'tss_distance', 'ma_samples', 'ma_count',
       'af', 'pval_nominal', 'slope', 'slope_se', 'pval_perm', 'pval_beta',
       'qval', 'pval_nominal_threshold', 'chr',
       'pos', 'strand', 'Name', 'AddressA', 'AddressB', 'ProbeSeqA',
       'ProbeSeqB', 'Type', 'NextBase', 'Color', 'Probe_rs', 'Probe_maf',
       'CpG_rs', 'CpG_maf', 'SBE_rs', 'SBE_maf', 'Islands_Name',
       'Relation_to_Island', 'Forward_Sequence', 'SourceSeq', 'Random_Loci',
       'Methyl27_Loci', 'UCSC_RefGene_Name', 'UCSC_RefGene_Accession',
       'UCSC_RefGene_Group', 'Phantom', 'DMR', 'Enhancer', 'HMM_Island',
       'Regulatory_Feature_Name', 'Regulatory_Feature_Group', 'DHS']
# Ensure these columns exist in your DataFrame

# Save the DataFrame with specified columns
filtered_results_annotated.to_csv(
    os.path.join(
        molQTL_loc, 'target_regions.nom_cis_mqtl_annotated.txt.gz'),
    sep='\t',
    index=False,
    compression='gzip',
    columns=columns_order
)


In [62]:
del filtered_results # remove the filtered_results DataFrame from memory
del filtered_results_annotated # remove the filtered_results_annotated DataFrame from memory
del filtered # remove the filtered DataFrame from memory
del filtered_results_pd # remove the filtered_results_pd DataFrame from memory
del merged_data # remove the merged_data DataFrame from memory
del annotation_data # remove the annotation_data DataFrame from memory